In [ ]:
# | default_exp _code_generator.plan_generator

In [ ]:
# | export

from typing import *
import time
import json

from yaspin import yaspin

from fastkafka._components.logger import get_logger
from fastkafka._code_generator.helper import CustomAIChat, ValidateAndFixResponse
from fastkafka._code_generator.prompts import PLAN_GENERATION_PROMPT

In [ ]:
from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

ENTITY_ERROR_MSG = {
    "invalid_entity": "The entities should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid entities",
    "invalid_name": "The name of the entity should be defined and cannot be empty. Please read the ==== APP DESCRIPTION: ==== and add a valid value to the 'name' key",
    "invalid_arguments": "The arguments of the entity should be a dictionary with key, value pairs and cannot be empty or any other datatype. Please read the ==== APP DESCRIPTION: ==== and generate valid arguments",
}

In [ ]:
# | export

def _validate_entities(plan: Dict[str, List[Dict[str, Any]]]) -> List[str]:
    """Validate the entities in the given plan and returns a list of any error messages encountered.

    Args:
        plan: The plan generated by OpenAI

    Returns:
        A list containing error messages for each validation failure. If there are no errors, an empty list is returned.
    """
    entities = plan.get("entities")
    if not isinstance(entities, list) or len(entities) == 0:
        return [ENTITY_ERROR_MSG["invalid_entity"]]

    errors = []
    for entity in entities:
        if not isinstance(entity.get("name"), str) or entity.get("name") == "":
            errors.append(ENTITY_ERROR_MSG["invalid_name"])
        if (
            not isinstance(entity.get("arguments"), dict)
            or entity.get("arguments") == {}
        ):
            errors.append(ENTITY_ERROR_MSG["invalid_arguments"])
    return errors

In [ ]:
fixture_plan = {
    "entities": [
        {
            "name": "entity 1",
            "arguments": {"name": "str"},
        },
        {
            "name": "entity 2",
            "arguments": {},
        }
    ]
}
expected = [ENTITY_ERROR_MSG["invalid_arguments"]]
actual = _validate_entities(fixture_plan)
print(actual)
assert actual == expected

['The arguments of the entity should be a dictionary with key, value pairs and cannot be empty or any other datatype. Please read the ==== APP DESCRIPTION: ==== and generate valid arguments']


In [ ]:
fixture_plan = {
    "entities": [
        {
            "name": "",
            "arguments": "",
        }
    ]
}
expected = [ENTITY_ERROR_MSG["invalid_name"], ENTITY_ERROR_MSG["invalid_arguments"]]
actual = _validate_entities(fixture_plan)
print(actual)
assert actual == expected

["The name of the entity should be defined and cannot be empty. Please read the ==== APP DESCRIPTION: ==== and add a valid value to the 'name' key", 'The arguments of the entity should be a dictionary with key, value pairs and cannot be empty or any other datatype. Please read the ==== APP DESCRIPTION: ==== and generate valid arguments']


In [ ]:
fixture_plan = {
    "entities": []
}
expected = [ENTITY_ERROR_MSG["invalid_entity"]]
actual = _validate_entities(fixture_plan)
print(actual)
assert actual == expected

['The entities should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid entities']


In [ ]:
fixture_plan = {
    "entities": {}
}
expected = [ENTITY_ERROR_MSG["invalid_entity"]]
actual = _validate_entities(fixture_plan)
print(actual)
assert actual == expected

['The entities should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid entities']


In [ ]:
fixture_plan = {
    "apps": []
}
expected = [ENTITY_ERROR_MSG["invalid_entity"]]
actual = _validate_entities(fixture_plan)
print(actual)
assert actual == expected

['The entities should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid entities']


In [ ]:
# | export

APPS_ERROR_MSG = {
    "invalid_app": "The apps should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid apps",
    "missing_app_keys": "The below keys are missing from the apps. Please read the ==== APP DESCRIPTION: ==== and add the missing keys",
    "invalid_app_name": "The app_name cannot have spaces. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid arguments",
    "invalid_kafka_brokers": "The kafka_brokers can either be a dictionary or None. It cannot have anyother data types. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid kafka_brokers",
}

CONSUME_FUNCTIONS_ERROR_MSG = {
    "invalid_functions": "The consumes_functions can either be a dictionary with key and value pairs or {}. It cannot have anyother data types. Please read the ==== APP DESCRIPTION: ==== and generate valid consumes_functions",
    "missing_functions_keys": "The below keys are missing from the '{}' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys",
    "invalid_prefix": "The '{}' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.\nConsume function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the consumes function in the ==== APP DESCRIPTION: ====, the default prefix 'on' should be used.",
}

PRODUCE_FUNCTIONS_ERROR_MSG = {
    "invalid_functions": "The produces_functions can either be a dictionary with key and value paris or {}. It cannot have anyother data types. Please read the ==== APP DESCRIPTION: ==== and generate valid produces_functions",
    "missing_functions_keys": "The below keys are missing from the '{}' produces_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys",
    "invalid_prefix": "The '{}' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.\nProduce function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the produces function, the default prefix 'to' should be used.",
    "missing_return": "The '{}' function has invalid return. The return key shoyuld have a value and it cannot be None. Please read the ==== APP DESCRIPTION: ==== and add a valid return type"
}

EXPECTED_FUNCTION_KEYS = [
    "topic",
    "prefix",
    "parameters",
]

In [ ]:
# | export


def _validate_for_missing_keys(
    key: str, missing_keys: List[str], errors: List[str], error_msgs: Dict[str, str]
) -> List[str]:
    """Validate for missing keys and append the error messages to the errors.

    Args:
        key: The key to be validated.
        missing_keys: List of missing keys to be appended.
        errors: List of existing errors to which new errors will be appended.
        error_msgs: Dictionary of common error messages.

    Returns:
        The updated list of errors after appending the missing keys error message.
    """
    missing_keys_error = error_msgs["missing_functions_keys"].format(key)
    missing_keys_list = "\n".join(sorted(missing_keys))
    errors.append(f"{missing_keys_error}\n\n{missing_keys_list}")

    return errors

In [ ]:
key = "sample_function"
missing_keys = ["key 1", "key 2"]
errors = []

expected = [CONSUME_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format(key) + "\n\n" + "\n".join(sorted(missing_keys))]
actual = _validate_for_missing_keys(key, missing_keys, errors, CONSUME_FUNCTIONS_ERROR_MSG)
print(actual)
assert actual == expected

["The below keys are missing from the 'sample_function' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys\n\nkey 1\nkey 2"]


In [ ]:
key = "sample_function"
missing_keys = ["key 3", "key 4"]
errors = ["key 1", "key 2"]

expected = ["key 1", "key 2", PRODUCE_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format(key) + "\n\n" + "\n".join(sorted(missing_keys))]
actual = _validate_for_missing_keys(key, missing_keys, errors, PRODUCE_FUNCTIONS_ERROR_MSG)
print(actual)
assert actual == expected

['key 1', 'key 2', "The below keys are missing from the 'sample_function' produces_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys\n\nkey 3\nkey 4"]


In [ ]:
# | export


def _validate_prefix(
    key: str,
    params: Dict[str, Union[str, List[Dict[str, str]]]],
    errors: List[str],
    error_msgs: Dict[str, str],
) -> List[str]:
    """Validate the prefix key in consumers/producers function.

    Args:
        key: The key to be validated.
        params: A dictionary containing the response from OpenAI.
        errors: A list of error messages.
        error_msgs: A dictionary containing common error messages.

    Returns:
        The updated list of error messages.
    """
    if key.split("_")[0] != params["prefix"]:
        errors.append(error_msgs["invalid_prefix"].format(key))
    return errors

In [ ]:
key = "on_sample_function"
params = {"prefix": "on"}
errors = []

expected = []
actual = _validate_prefix(key, params, errors, CONSUME_FUNCTIONS_ERROR_MSG)
print(actual)
assert actual == expected

[]


In [ ]:
key = "to_sample_function"
params = {"prefix": "on"}
errors = []

expected = [CONSUME_FUNCTIONS_ERROR_MSG["invalid_prefix"].format(key)]
actual = _validate_prefix(key, params, errors, CONSUME_FUNCTIONS_ERROR_MSG)
print(actual)
assert actual == expected

["The 'to_sample_function' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.\nConsume function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the consumes function in the ==== APP DESCRIPTION: ====, the default prefix 'on' should be used."]


In [ ]:
key = "to_sample_function"
params = {"prefix": "on"}
errors = []

expected = [PRODUCE_FUNCTIONS_ERROR_MSG["invalid_prefix"].format(key)]
actual = _validate_prefix(key, params, errors, PRODUCE_FUNCTIONS_ERROR_MSG)
print(actual)
assert actual == expected

["The 'to_sample_function' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.\nProduce function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the produces function, the default prefix 'to' should be used."]


In [ ]:
# | export


def _get_error_msgs_and_expected_keys(
    is_producer_function: bool,
) -> Tuple[Dict[str, str], List[str]]:
    """Get appropriate error messages and expected keys to be checked for the given function.

    Args:
        is_producer_function: Flag indicating whether the function is a producer function or not.

    Returns:
        A tuple containing a dictionary of error messages and a list of expected keys.
    """
    if is_producer_function:
        return PRODUCE_FUNCTIONS_ERROR_MSG, EXPECTED_FUNCTION_KEYS + ["returns"]
    else:
        return CONSUME_FUNCTIONS_ERROR_MSG, EXPECTED_FUNCTION_KEYS

In [ ]:
error_msgs,expected_keys =  _get_error_msgs_and_expected_keys(False)

print(error_msgs)
assert error_msgs == CONSUME_FUNCTIONS_ERROR_MSG

print(expected_keys)
assert expected_keys == EXPECTED_FUNCTION_KEYS

{'invalid_functions': 'The consumes_functions can either be a dictionary with key and value pairs or {}. It cannot have anyother data types. Please read the ==== APP DESCRIPTION: ==== and generate valid consumes_functions', 'missing_functions_keys': "The below keys are missing from the '{}' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys", 'invalid_prefix': "The '{}' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.\nConsume function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the consumes function in the ==== APP DESCRIPTION: ====, the default prefix 'on' should be used."}
['topic', 'prefix', 'parameters']


In [ ]:
error_msgs,expected_keys =  _get_error_msgs_and_expected_keys(True)

print(error_msgs)
assert error_msgs == PRODUCE_FUNCTIONS_ERROR_MSG

print(expected_keys)
assert expected_keys == EXPECTED_FUNCTION_KEYS + ["returns"]

{'invalid_functions': 'The produces_functions can either be a dictionary with key and value paris or {}. It cannot have anyother data types. Please read the ==== APP DESCRIPTION: ==== and generate valid produces_functions', 'missing_functions_keys': "The below keys are missing from the '{}' produces_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys", 'invalid_prefix': "The '{}' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.\nProduce function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the produces function, the default prefix 'to' should be used.", 'missing_return': "The '{}' function has invalid return. The return key shoyuld have a value and it cannot be None. Please read the ==== APP DESCRIPTION: ==== and add a valid return type"}
['topic', 'prefix', 'parameters', 'returns']


In [ ]:
# | export


def _validate_functions(
    functions: Dict[str, Dict[str, Union[str, List[Dict[str, str]]]]],
    errors: List[str],
    is_producer_function: bool = False,
) -> List[str]:
    """Validate the given functions dictionary

    Args:
        functions: A dictionary containing function names as keys and their properties as values.
        errors: A list of error messages.
        is_producer_function: A flag indicating whether the functions to be validated are producer functions. Defaults to False.

    Returns:
        A list of error messages. If no errors are found, an empty list is returned.
    """
    error_msgs, expected_keys = _get_error_msgs_and_expected_keys(is_producer_function)

    if not isinstance(functions, dict):
        errors.append(error_msgs["invalid_functions"])
        return errors

    if functions == {}:
        return errors

    for key, params in functions.items():
        missing_keys = list(set(expected_keys) - set(params.keys()))
        if len(missing_keys) > 0:
            errors = _validate_for_missing_keys(key, missing_keys, errors, error_msgs)
        else:
            errors = _validate_prefix(key, params, errors, error_msgs)
            if is_producer_function:
                if str(params["returns"]) == "None":
                    errors.append(error_msgs["missing_return"].format(key))
    return errors

In [ ]:
fixture_produces_functions = {
    "to_store_product": {
        "topic": "store_product",
        "prefix": "to",
        "parameters": [{"store_product": "StoreProduct"}],
        "returns": "None",
    }
}
expected = [PRODUCE_FUNCTIONS_ERROR_MSG["missing_return"].format("to_store_product")]
actual = _validate_functions(fixture_produces_functions, [], True)
print(actual)
assert actual == expected

["The 'to_store_product' function has invalid return. The return key shoyuld have a value and it cannot be None. Please read the ==== APP DESCRIPTION: ==== and add a valid return type"]


In [ ]:
fixture_consumes_functions = {
    "on_store_product": {
        "topic": "store_product",
        "prefix": "on",
        "parameters": {"name": "str"},
    },
    "to_buy_product": {
        "topic": "share_product",
        "prefix": "on",
        "parameters": {"name": "str"},
    },
}

expected = [CONSUME_FUNCTIONS_ERROR_MSG["invalid_prefix"].format("to_buy_product")]
actual = _validate_functions(fixture_consumes_functions, [])
for a in actual:
    print(f"{a}\n\n")
assert actual == expected

The 'to_buy_product' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.
Consume function names should follow the format: prefix + '_' + topic name. If the user doesn't explicitly define the prefix for the consumes function in the ==== APP DESCRIPTION: ====, the default prefix 'on' should be used.




In [ ]:
fixture_consumes_functions = {
    "on_store_product": {
        "topic": "store_product",
        "prefix": "on",
    }
}

missing_consumes_function_keys = ["parameters", "returns"]
expected = [
    PRODUCE_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format(
        "on_store_product"
    )
    + "\n\n"
    + "\n".join(sorted(missing_consumes_function_keys))
]
actual = _validate_functions(fixture_consumes_functions, [], True)
print(actual[0])
assert actual == expected

The below keys are missing from the 'on_store_product' produces_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

parameters
returns


In [ ]:
fixture_consumes_functions = {
    "on_store_product": {
        "topic": "store_product",
        "prefix": "on",
    },
    "on_buy_product": {
        "topic": "store_product",
        "parameters": {"name": "str"}
    }
}

expected = [
    CONSUME_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format(
        "on_store_product"
    )
    + "\n\n"
    + "parameters",
    CONSUME_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format(
        "on_buy_product"
    )
    + "\n\n"
    + "prefix"
]
actual = _validate_functions(fixture_consumes_functions, [], False)
for a in actual:
    print(f"{a}\n\n") 
assert actual == expected

The below keys are missing from the 'on_store_product' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

parameters


The below keys are missing from the 'on_buy_product' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

prefix




In [ ]:
fixture_consumes_functions = {}

actual = _validate_functions(fixture_consumes_functions, [])
print(actual)
assert actual == []

[]


In [ ]:
fixture_consumes_functions = {
    "on_store_product": {
        "topic": "store_product",
        "prefix": "on",
        "parameters": [{"msg": "StoreProduct"}],
    }
}

actual = _validate_functions(fixture_consumes_functions, [])
print(actual)
assert actual == []

[]


In [ ]:
# | export

EXPECTED_APP_KEYS = [
    "app_name",
    "kafka_brokers",
    "title",
    "consumes_functions",
    "produces_functions",
]


def _validate_apps(plan: Dict[str, List[Dict[str, Any]]]) -> List[str]:
    """Validate the 'apps' part of the generated plan.

    Args:
        plan: The plan generated by OpenAI

    Returns:
        A list of error messages if there are any errors, otherwise an empty list.
    """
    apps = plan.get("apps")
    if not isinstance(apps, list) or len(apps) == 0:
        return [APPS_ERROR_MSG["invalid_app"]]

    errors = []
    for app in apps:
        missing_app_keys = list(set(EXPECTED_APP_KEYS) - set(app.keys()))
        if len(missing_app_keys) > 0:
            return [
                APPS_ERROR_MSG["missing_app_keys"]
                + "\n\n"
                + "\n".join(sorted(missing_app_keys))
            ]
        else:
            if len(app["app_name"].split(" ")) != 1:
                errors.append(APPS_ERROR_MSG["invalid_app_name"])
            if (
                not isinstance(app["kafka_brokers"], dict)
                and not str(app["kafka_brokers"]) == "None"
            ):
                errors.append(APPS_ERROR_MSG["invalid_kafka_brokers"])
            for func_details, flag in [
                (app["consumes_functions"], False),
                (app["produces_functions"], True),
            ]:
                errors = _validate_functions(func_details, errors, flag)
    return errors

In [ ]:
fixture_plan = {
    "apps": [{
        "app_name": "my_app_name",
        "kafka_brokers": "invalid kafka_brokers",
        "title": "some title",
        "consumes_functions": {},
        "produces_functions": {}
    }]
}
expected = [APPS_ERROR_MSG["invalid_kafka_brokers"]]
actual = _validate_apps(fixture_plan)
print(actual)
assert actual == expected

["The kafka_brokers can either be a dictionary or None. It cannot have anyother data types. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid kafka_brokers"]


In [ ]:
fixture_plan = {
    "apps": [{
        "app_name": "my_app_name",
        "kafka_brokers": "None",
        "title": "some title",
        "consumes_functions": {},
        "produces_functions": {}
    }]
}
expected = []
actual = _validate_apps(fixture_plan)
print(actual)
assert actual == expected

[]


In [ ]:
fixture_plan = {
    "apps": [
        {
            "app_name": "my app name",
            "kafka_brokers": "invalid kafka_brokers",
            "title": "some title",
            "consumes_functions": {
                "on_store_product": {
                    "topic": "store_product",
                    "prefix": "on",
                },
                "on_buy_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                },
            },
            "produces_functions": {
                "to_sell_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                    "prefix": "to",
                },
                "on_buy_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                    "prefix": "to",
                    "returns": "SomeClass"
                },
                "to_recall_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                    "prefix": "to",
                    "returns": "None"
                },
            },
        }
    ]
}
missing_app_keys = ["produces_functions", "consumes_functions", "title"]
expected = [
    APPS_ERROR_MSG["invalid_app_name"],
    APPS_ERROR_MSG["invalid_kafka_brokers"],
    CONSUME_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format("on_store_product") + "\n\nparameters",
    CONSUME_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format("on_buy_product") + "\n\nprefix",
    PRODUCE_FUNCTIONS_ERROR_MSG["missing_functions_keys"].format("to_sell_product") + "\n\nreturns",
    PRODUCE_FUNCTIONS_ERROR_MSG["invalid_prefix"].format("on_buy_product"),
    PRODUCE_FUNCTIONS_ERROR_MSG["missing_return"].format("to_recall_product"),
    
]
actual = _validate_apps(fixture_plan)
for a in actual:
    print(a)
assert actual == expected

The app_name cannot have spaces. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid arguments
The kafka_brokers can either be a dictionary or None. It cannot have anyother data types. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid kafka_brokers
The below keys are missing from the 'on_store_product' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

parameters
The below keys are missing from the 'on_buy_product' consumes_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

prefix
The below keys are missing from the 'to_sell_product' produces_functions. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

returns
The 'on_buy_product' funtion name is having invalid prefix in the name. Please fix the function name using the following rule.
Produce funct

In [ ]:
fixture_plan = {
    "apps": [
        {
            "app_name": "my app name",
            "kafka_brokers": "invalid kafka_brokers",
            "title": "some title",
            "consumes_functions": {},
            "produces_functions": {},
        }
    ]
}
missing_app_keys = ["produces_functions", "consumes_functions", "title"]
expected = [APPS_ERROR_MSG["invalid_app_name"], APPS_ERROR_MSG["invalid_kafka_brokers"]]
actual = _validate_apps(fixture_plan)
print(actual)
assert actual == expected

["The app_name cannot have spaces. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid arguments", "The kafka_brokers can either be a dictionary or None. It cannot have anyother data types. The app_name should be in lower letters and can have 'underscore'. Please read the ==== APP DESCRIPTION: ==== and generate valid kafka_brokers"]


In [ ]:
fixture_plan = {
    "apps": [{
        "app_name": "my app name",
        "kafka_brokers": None,
    }]
}
missing_app_keys = ["produces_functions", "consumes_functions", "title"]
expected = [APPS_ERROR_MSG["missing_app_keys"] + "\n\n" + "\n".join(sorted(missing_app_keys))]
actual = _validate_apps(fixture_plan)
print(actual[0])
assert actual == expected

The below keys are missing from the apps. Please read the ==== APP DESCRIPTION: ==== and add the missing keys

consumes_functions
produces_functions
title


In [ ]:
fixture_plan = {
    "apps": []
}
expected = [APPS_ERROR_MSG["invalid_app"]]
actual = _validate_apps(fixture_plan)
print(actual)
assert actual == expected

['The apps should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid apps']


In [ ]:
fixture_plan = {
    "entities": []
}
expected = [APPS_ERROR_MSG["invalid_app"]]
actual = _validate_apps(fixture_plan)
print(actual)
assert actual == expected

['The apps should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid apps']


In [ ]:
# | export

def _vaidate_plan(plan: Dict[str, List[Dict[str, Any]]]) -> List[str]:
    """Validates the generated plan

    Args:
        plan: The plan to be validated.

    Returns:
        A list of error messages generated during the validation process. If no errors are found, an empty list is returned.
    """
    entity_error = _validate_entities(plan)
    app_error = _validate_apps(plan)
    return entity_error + app_error

In [ ]:
fixture_plan = {"entities": [], "apps": []}
expected = [ENTITY_ERROR_MSG["invalid_entity"], APPS_ERROR_MSG["invalid_app"]]

actual = _vaidate_plan(fixture_plan)
print(actual)

assert actual == expected

['The entities should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid entities', 'The apps should be a list and cannot be empty in the generated plan. Please read the ==== APP DESCRIPTION: ==== and generate valid apps']


In [ ]:
fixture_plan = {
    "entities": [
        {
            "name": "entity 1",
            "arguments": {"name": "str"},
        },
        {
            "name": "entity 2",
            "arguments": {"name": "str"},
        },
    ],
    "apps": [
        {
            "app_name": "my_app_name",
            "kafka_brokers": "None",
            "title": "some title",
            "consumes_functions": {
                "on_store_product": {
                    "topic": "store_product",
                    "prefix": "on",
                    "parameters": {"name": "str"},
                },
                "on_buy_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                    "prefix": "on",
                },
            },
            "produces_functions": {
                "to_sell_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                    "prefix": "to",
                    "returns": "SomeClass",
                },
                "to_buy_product": {
                    "topic": "store_product",
                    "parameters": {"name": "str"},
                    "prefix": "to",
                    "returns": "SomeClass",
                },
            },
        }
    ],
}
expected = []

actual = _vaidate_plan(fixture_plan)
print(actual)

assert actual == expected

[]


In [ ]:
# | export

def _validate_response(response: str) -> List[str]:
    """Validate the plan response generated by OpenAI

    Args:
        response: The JSON plan response generated by OpenAI in string format.

    Returns:
        Returns a list of errors if any found during the validation of the plan.

    Raises:
        json.JSONDecodeError: If the response is not a valid JSON.
    """
    try:
        response_dict = json.loads(response)
        errors_list = _vaidate_plan(response_dict)
        return errors_list
    except json.JSONDecodeError as e:
        return ["JSON decoding failed. Please send JSON response only."]

In [ ]:
response = """
invalid json string
"""

_validate_response(response)

['JSON decoding failed. Please send JSON response only.']

In [ ]:
response = """{
  "entities": [
    {
      "name": "StoreProduct",
      "arguments": {
        "product_name": "str",
        "currency": "str",
        "price": "float"
      }
    }
  ],
  "apps": [
    {
      "app_name": "store_app",
      "kafka_brokers": {
        "localhost": {
          "url": "localhost",
          "description": "local development kafka broker",
          "port": 9092
        }
      },
      "title": "Store Kafka App",
      "consumes_functions": {
        "on_store_product": {
          "topic": "store_product",
          "prefix": "on",
          "parameters": {
            "msg": "StoreProduct"
          },
          "description": "This function will listen to the 'store_product' topic, it will consume the messages posted on the 'store_product' topic. The message should be of type 'StoreProduct' which contains the attributes 'product_name', 'currency', and 'price'. After consuming the data, it will forward the store product details to the 'change_currency' topic."
        }
      },
      "produces_functions": {
        "to_change_currency": {
          "topic": "change_currency",
          "prefix": "to",
          "parameters": {
            "store_product": "StoreProduct"
          },
          "description": "This function will be triggered when a store product is received from the 'store_product' topic. It will take the store product as input and will produce a message to the 'change_currency' topic. If the currency in the input store product is 'HRK', the currency will be set to 'EUR', and the price will be divided by 7.5. After producing the message, it will return the transformed store product.",
          "returns": "StoreProduct"
        }
      }
    }
  ]
}"""

expected = []
actual = _validate_response(response)
print(actual)
assert actual == expected

[]


In [ ]:
# | export


def generate_plan(description: str) -> Tuple[str, str]:
    """Generate a plan from user's application description

    Args:
        description: Validated User application description

    Returns:
        The plan generated by OpenAI as a dictionary
    """
    with yaspin(
        text="Generating plan", #  (slowest step, usually takes 30 to 90 seconds)...
        color="cyan",
        spinner="clock",
    ) as sp:
        plan_generator = CustomAIChat(user_prompt=PLAN_GENERATION_PROMPT)
        plan_validator = ValidateAndFixResponse(plan_generator, _validate_response)
        validated_plan, total_tokens = plan_validator.fix(description)
        
        sp.text = ""
        sp.ok(" ✔ Plan generated")
        return validated_plan, total_tokens

In [ ]:
app_description = """
Create FastKafka application which consumes messages from the store_product topic, it consumes messages with three attributes: product_name, currency and price. While consuming, it should produce a message to the change_currency topic. input parameters for this producing function should be store_product object and function should return store_product. produces function should check if the currency in the input store_product parameter is "HRK", currency should be set to "EUR" and the price should be divided with 7.5.
app should use localhost broker
"""
plan, total_tokens = generate_plan(app_description)
print(json.loads(plan))
assert int(total_tokens) > 0
print(total_tokens)

⠹ Generating plan 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:59: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ Plan generated 
{'entities': [{'name': 'StoreProduct', 'arguments': {'product_name': 'str', 'currency': 'str', 'price': 'float'}}], 'apps': [{'app_name': 'store_app', 'kafka_brokers': {'localhost': {'url': 'localhost', 'description': 'local development kafka broker', 'port': 9092}}, 'title': 'Store Kafka App', 'consumes_functions': {'on_store_product': {'topic': 'store_product', 'prefix': 'on', 'parameters': {'msg': 'StoreProduct'}, 'description': "This function will listen to the 'store_product' topic, it will consume the messages posted on the 'store_product' topic. The message should be of type 'StoreProduct' which contains product details such as 'product_name', 'currency', and 'price'. After consuming the data, it will produce a message to the 'change_currency' topic."}}, 'produces_functions': {'to_change_currency': {'topic': 'change_currency', 'prefix': 'to', 'parameters': {'store_product': 'StoreProduct'}, 'description': "This function will be triggered when a message is rece